<h1 style="color:#0088ED;text-align:center"> Análise exploratória de Dados</h1>

Neste notebook, vamos explorar um conjunto de dados clientes de uma empresa do setor financeiro. O objetivo é entender o perfil dos clientes que possam ser mais propensos a abandonar o serviço oferecido de ERP (Enterprise Resource Planning) e, com isso, elaborar estratégias para reter esses clientes.

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
from matplotlib import patches as mpatches
from textwrap import wrap
import seaborn as sns
import pandas_gbq

from IPython.display import display, HTML


%matplotlib inline

In [ ]:
sheet_id = "1qU9oeYgrrpXGAiqiB9jZ_uQgoXxEuRLVU9iPj83UTkw"
sheet_name = "customer_churn_data"
df = pd.read_csv(
    f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
)

In [ ]:
df.columns = (
    df.columns.str.lower()
    .str.strip()
    .str.replace(" ", "_")
    .str.normalize("NFKD")
    .str.encode("ascii", errors="ignore")
    .str.decode("utf-8")
    .str.split(":")
    .str[-1]
    .str.replace(".", "__")
)

## 1. Visão geral do conjunto de dados

O conjunto de dados é composto por 22 colunas e 7043 linhas, sendo `Churn` a variável dependente que desejamos prever. A seguir, uma visão geral das colunas do conjunto de dados:

- **ID** : identificação do cliente
- **Tipo de empresa**: tipo de empresa, sendo "Pequena empresa" ou "Micro empresa". 
- **Fundação da empresa**: Ano de fundação da empresa (de 2001 a 2021)
- **Possui mais de um sócio**: Se a empresa possui mais de um sócio - Sim ou Não
- **Funcionários**: Número de funcionários - "até 5 funcionários" ou "6 ou mais funcionários"
- **Meses de permanência**: quantidade de meses que o cliente permaneceu com o serviço
- **Utiliza serviços financeiros**: Se o cliente utiliza serviços financeiros - Sim ou Não
- **PossuiContador**: Se o cliente possui contador - Sim ou Não
- **Faz conciliação bancária**: Tipo de conciliação bancária - "manual" ou "automática" ou "não faz"
- **Frequência de utilização de feature do sistema:**
    - Módulo financeiro: "Pouco uso", "Uso frequente" ou "Nunca utilizou"
    - Emissão de nota fiscal: "Pouco uso", "Uso frequente" ou "Nunca utilizou"
    - Integração bancária: "Pouco uso", "Uso frequente" ou "Nunca utilizou"
    - Módulo de vendas: "Pouco uso", "Uso frequente" ou "Nunca utilizou"
    - Relatórios: "Pouco uso", "Uso frequente" ou "Nunca utilizou"
    - Utilização de APIs de integração: "Pouco uso", "Uso frequente" ou "Nunca utilizou"
- **Contrato**: Tipo de contrato - "Mês-a-mês", "Anual" ou "Trimestral"
- **Emite boletos**: Se emite boletos - 0 ou 1
- **Emite boletos.1**: Se emite boletos - Yes ou No
- **Tipo de pagamento**: Forma usada para pagamento e frequência - "Boleto - pagamento único", "Boleto - mês a mês", "Cartão de crédito - pagamento único" ou "Cartão de crédito - mês a mês"
- **Receita mensal**: Receita mensal do cliente
- **Receita total**: Receita total do cliente
- **Churn**: Se o cliente abandonou o serviço - Sim ou Não



In [ ]:
df.shape, df["id"].nunique()

In [ ]:
df = df.set_index("id")

In [ ]:
# checar duplicatas
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.info()

In [ ]:
df["possuicontador"].isna().sum(), df["receita_total"].isna().sum()

Todas as colunas possuem dados preenchidos, exceto as colunas `possuicontador`, onde 682 não estão preenchidos e `receita total` com 11 faltantes. A razão provável para isso pode ser oriúnda de vários fatores como:

- O cliente não quis informar.
- O atributo poderia não existir na época em que os dados foram coletados.
- Distração do preenchimento do formulário.
- Problemas técnicos na coleta de dados.
- A informação não era conhecida na época da coleta de dados.


Estratégias que podem ser adotadas para lidar com os valores nulos em `possuicontador` são:

- Preencher os valores nulos com a moda da coluna.
- Usar um estimador para preencher os valores nulos.
- Preencher os valores nulos com "Não informado", assumindo que o cliente não quis informar ou a informação não era conhecida na época da coleta de dados.
- Preencher os valores nulos com "Não" se considerarmos que o cliente não possui contador.

Algumas estratégias que podem ser adotadas para lidar com os valores nulos em `receita total` são:

- Preencher os valores nulos com a média, mediana ou moda da coluna.
- Usar um estimador para preencher os valores nulos.
- Preencher com a receita mensal, assumindo que a receita total é igual a receita mensal.

In [ ]:
msno.matrix(df, color=(62 / 255, 143 / 255, 196 / 255))

In [ ]:
with pd.option_context("display.max_columns", None, "display.max_rows", 10):
    display(df)

### 1.1 Pré-processamento dos dados
As variáveis `Receita mensal` e `Receita total` podem ser convertidas para o tipo `float` para facilitar a análise, dado que ambas se referem a valores monetários.

In [ ]:
def convert_to_float(amount):
    return (
        amount.str.replace("R$", "")
        .str.replace(".", "|")
        .str.replace(",", ".")
        .str.replace("|", "")
        .astype(float)
    )

In [ ]:
df["receita_mensal"] = convert_to_float(df["receita_mensal"])
df["receita_total"] = convert_to_float(df["receita_total"])

A variável `Emite boletos.1` possui a mesma informação que a variável `Emite boletos`, logo, uma delas pode ser removida.

In [ ]:
df["emite_boletos__1"].replace({"Yes": "1", "No": "0"}).astype(int).equals(
    df["emite_boletos"]
)

In [ ]:
df = df.drop("emite_boletos__1", axis=1, errors="ignore")

A variável `Tipo de pagamento` pode ser dividida em duas variáveis: `Forma de pagamento` e `Frequência de pagamento`. 

In [ ]:
df[["forma_de_pagamento", "frequencia_de_pagamento"]] = df[
    "tipo_de_pagamento"
].str.split(" - ", expand=True)
df = df.drop("tipo_de_pagamento", axis=1)

Algumas variáveis do conjunto são do tipo qualitativa ordinal, ou seja, possuem uma ordem natural. Por exemplo, a variável `Frequência de utilização de feature do sistema: Módulo financeiro` possui a ordem "Nunca utilizou" < "Pouco uso" < "Uso frequente". Desse modo, transformamos essas variáveis em categorias ordinais.

In [ ]:
df["funcionarios"].unique(), df["_modulo_financeiro"].unique(), df[
    "contrato"
].unique(), df["frequencia_de_pagamento"].unique()

In [ ]:
def categorize_column(df, column_name):
    categories = ["Nunca utilizou", "Pouco uso", "Uso frequente"]
    return pd.Categorical(df[column_name], categories=categories, ordered=True)


df["funcionarios"] = pd.Categorical(
    df["funcionarios"],
    categories=["até 5 funcionários", "6 ou mais funcionários"],
    ordered=True,
)

df["contrato"] = pd.Categorical(
    df["contrato"],
    categories=["Mês-a-mês", "Trimestral", "Anual"],
    ordered=True,
)

df["frequencia_de_pagamento"] = pd.Categorical(
    df["frequencia_de_pagamento"],
    categories=["mês a mês", "pagamento único"],
    ordered=True,
)

df["_emissao_de_nota_fiscal"] = categorize_column(df, "_emissao_de_nota_fiscal")
df["_integracao_bancaria"] = categorize_column(df, "_integracao_bancaria")
df["_modulo_de_vendas"] = categorize_column(df, "_modulo_de_vendas")
df["_modulo_financeiro"] = categorize_column(df, "_modulo_financeiro")
df["_relatorios"] = categorize_column(df, "_relatorios")
df["_utilizacao_de_apis_de_integracao"] = categorize_column(
    df, "_utilizacao_de_apis_de_integracao"
)

df["tipo_de_empresa"] = pd.Categorical(
    df["tipo_de_empresa"],
    categories=[
        "Micro empresa",
        "Pequena empresa",
    ],
)
df["faz_conciliacao_bancaria"] = df["faz_conciliacao_bancaria"].astype("category")
df["forma_de_pagamento"] = df["forma_de_pagamento"].astype("category")
df["possui_mais_de_um_socio"] = pd.Categorical(
    df["possui_mais_de_um_socio"],
    categories=[
        "Não",
        "Sim",
    ],
)

df["utiliza_servicos_financeiros"] = pd.Categorical(
    df["utiliza_servicos_financeiros"],
    categories=[
        "Não",
        "Sim",
    ],
)

df["churn"] = pd.Categorical(
    df["churn"],
    categories=[
        "Não",
        "Sim",
    ],
)

df["emite_boletos"] = pd.Categorical(
    df["emite_boletos"].astype(str).replace({"1": "Sim", "0": "Não"}),
    categories=[
        "Não",
        "Sim",
    ],
)

### 1.2 Análise descritiva
A análise descritiva é uma etapa importante para entender o comportamento das variáveis do conjunto de dados. Ela nos permite identificar possíveis problemas nos dados, como outliers, valores faltantes, distribuição das variáveis, entre outros.

In [ ]:
df.hist(bins=20, edgecolor="black", grid=False)
plt.show()

Usando o método `describe`, podemos obter estatísticas descritivas das variáveis numéricas. Vemos que o cliente que permaneceu por mais tempo com o serviço ficou 72 meses, enquanto o que permaneceu por menos tempo menos de 1 mês. A receita mensal média é de R\$ 64.79 e a receita total média é de R\$ 2.285,54. A maior parte das empresas foi fundada a partir de 2016, sendo que 50% das empresas possuem receita total de até R\$ 1.399,35

In [ ]:
desc = df.describe().T
desc["skewness"] = df.select_dtypes(include="number").skew().round(2)
desc["kurtosis"] = df.select_dtypes(include="number").kurt().round(2)
desc

A maior parte das empresas do conjunto é do tipo "Pequena empresa", não possui mais de um sócio, possui até 5 funcionários, utiliza serviços financeiros, faz pouco uso dos sistema, tem contrato mês-a-mês e emite boletos com frequência de pagamento única. E como esperado, a maioria dos clientes não abandonou o serviço. O campo `utiliza serviços financeiros` é o campo mais desbalanceado, composto por 6.354 registros "Sim" do total de 7.043 registros.

In [ ]:
df.describe(include=["category", "object"])

Com relação ao uso do sistema, vemos que o percentual de 21,7% que nunca usou aparece em todas as *features* do sistema. Dos clientes que usam o sistema, a maioria faz pouco uso. No entanto, quanto vemos cada módulo particular, notamos que a utilização de APIs de integração é a que tem mais uso frequente, seguida pelo módulo de relatórios.

In [ ]:
system_features = [
    "_modulo_financeiro",
    "_emissao_de_nota_fiscal",
    "_integracao_bancaria",
    "_modulo_de_vendas",
    "_relatorios",
    "_utilizacao_de_apis_de_integracao",
]

fig, ax = plt.subplots(2, 3, figsize=(20, 10))
fig.suptitle("Sistema")
for i, feature in enumerate(system_features):
    df[feature].value_counts().plot.pie(
        autopct="%1.1f%%",
        ax=ax[i // 3, i % 3],
        colors=["#3E8FC4", "#07478E", "#D3E3F3"],
    )
    ax[i // 3, i % 3].set_title(feature)
    ax[i // 3, i % 3].set_ylabel("")
plt.show()

Com relação a variável churn:

In [ ]:
# bar plot churn

churn_counts = df["churn"].value_counts(normalize=True).round(4).to_frame().T


fig, ax = plt.subplots(figsize=(8, 2))
churn_counts.plot(
    kind="barh", stacked=True, ax=ax, color=["#3E8FC4", "#07478E"], width=2
)
ax.set_xlim(0, 1)
ax.set_xticks([])
ax.set_yticks([])


for spine in ax.spines.values():
    spine.set_visible(False)

ax.get_legend().remove()

ax.annotate(
    f"{churn_counts['Sim'].values[0]*100}%",
    xy=(1 - churn_counts["Sim"].values[0] / 2, 0),
    va="center",
    ha="center",
    fontsize=30,
    fontweight="normal",
    fontfamily="serif",
    color="white",
)

ax.annotate(
    f"{churn_counts['Não'].values[0]*100}%",
    xy=(churn_counts["Não"].values[0] / 2, 0),
    va="center",
    ha="center",
    fontsize=30,
    fontweight="normal",
    fontfamily="serif",
    color="white",
)

ax.annotate(
    "Churn",
    xy=(0.95 - churn_counts["Sim"].values[0] / 2, -0.75),
    va="center",
    fontsize=16,
    fontweight="light",
    fontfamily="serif",
    color="white",
)

ax.annotate(
    "Não Churn",
    xy=(churn_counts["Não"].values[0] / 2 - 0.1, -0.75),
    va="center",
    fontsize=16,
    fontweight="light",
    fontfamily="serif",
    color="white",
)

fig.text(
    0.125,
    1.1,
    "Distribuição de Churn dos Clientes",
    fontsize=18,
    fontweight="bold",
    fontfamily="serif",
    color="#323232",
)

fig.text(
    0.125,
    0.95,
    "Dos 7.036 clientes, 26,5% cancelaram o serviço",
    fontsize=12,
    fontweight="light",
    fontfamily="serif",
    color="#323232",
)


plt.show()

## 2. Análise exploratória de dados

Nessa seção, vamos explorar as relações entre as variáveis do conjunto de dados. Vamos começar com a análise das variáveis categóricas e, em seguida, analisar as variáveis numéricas.


In [ ]:
def plot_categorical_feature(df, columns, axes):
    for ax, column in zip(axes.flatten(), columns):
        grouped = df.groupby([column, "churn"], observed=True).size()

        (grouped / grouped.groupby(level=0, observed=True).sum()).unstack().plot.bar(
            stacked=True,
            color=["#c3e3f7", "#3E8FC4"],
            ax=ax,
            width=1,
            edgecolor="white",
        )

        ax.set_title(column)
        ax.set_ylabel("")

        ax.set_xticklabels(ax.get_xticklabels(), rotation=0)

        # inserir porcentagem dentro das barras
        for p in ax.patches:
            width = p.get_width()
            height = p.get_height()
            x, y = p.get_xy()
            ax.text(
                x + width / 2,
                y + height / 2,
                f"{height * 100:.2f}%",
                horizontalalignment="center",
                verticalalignment="center",
            )

            ax.spines[["top", "right"]].set_visible(False)
            ax.set_xlabel("")

            ax.set_yticks([0, 0.25, 0.5, 0.75, 1])
            ax.set_yticklabels(["0%", "25%", "50%", "75%", "100%"])
            ax.set_xticklabels(
                [
                    ("\n".join(wrap(label.get_text(), 15)))
                    for label in ax.get_xticklabels()
                ]
            )
        ax.get_legend().remove()

    plt.legend(
        handles=[
            mpatches.Patch(color="#c3e3f7", label="Renovou"),
            mpatches.Patch(color="#3E8FC4", label="Não renovou"),
        ],
        bbox_to_anchor=(1, 1.5),
    )

    plt.tight_layout()
    plt.show()


_, axes = plt.subplots(2, 3, figsize=(16, 8))
plot_categorical_feature(
    df,
    [
        "_emissao_de_nota_fiscal",
        "_integracao_bancaria",
        "_modulo_de_vendas",
        "_modulo_financeiro",
        "_relatorios",
        "_utilizacao_de_apis_de_integracao",
    ],
    axes,
)

Nos casos em que houveram *churning*, notamos que os módulos com "pouco uso" são os que mais aparecem. No entanto, vemos que o percentual de clientes que "nunca utilizou" é maior para os clientes que renovaram o contrato. 

In [ ]:
_, axes = plt.subplots(2, 5, figsize=(18, 6))
plot_categorical_feature(
    df,
    [
        "tipo_de_empresa",
        "possui_mais_de_um_socio",
        "funcionarios",
        "utiliza_servicos_financeiros",
        "faz_conciliacao_bancaria",
        "contrato",
        "emite_boletos",
        "forma_de_pagamento",
        "frequencia_de_pagamento",
        "possuicontador",
    ],
    axes,
)

Relacionando a variável de churn com as demais variáveis nominais, vemos que os clientes que fizeram churn, em sua maioria, faz contrato mês-a-mês e o tipo de conciliação bancária é "manual" com pagamento único via boleto.

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(18, 3))

for ax, column in zip(axes.flatten(), df.select_dtypes(include="number").columns):
    for churn, color in zip(df["churn"].cat.categories, ["#c3e3f7", "#3E8FC4"]):
        df[df["churn"] == churn][column].plot.hist(
            ax=ax, alpha=0.5, color=color, bins=20, label=churn
        )
    ax.set_title(column)
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.spines[["top", "right"]].set_visible(False)


plt.legend(
    handles=[
        mpatches.Patch(color="#c3e3f7", label="Renovou"),
        mpatches.Patch(color="#3E8FC4", label="Não renovou"),
    ],
    bbox_to_anchor=(1.5, 1),
)

plt.suptitle("")
plt.tight_layout()
plt.show()

Ao analisar as variáveis numéricas, observamos que a distribuição de meses de permanência tem assimetria diferente para os clientes que deram *churn* e os que não deram. Clientes que renovaram tinham uma distribuição mais distribuída, enquanto os que fizeram churn têm uma distribuição mais assimétrica à direita, ou seja, a maioria dos clientes que saíu permaneceu com o serviço por um curto período de tempo. Com relação a receita mensal, notamos que clientes com maior receita foram os que fizeram churn.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 4))

df.boxplot(column="meses_de_permanencia", by="churn", ax=axes[0])
df.boxplot(column="receita_mensal", by="churn", ax=axes[1])

axes[0].set_title("Meses de Permanência")
axes[0].set_xlabel("")

axes[1].set_title("Receita Mensal")
axes[1].set_xlabel("")
plt.suptitle("")

# remover grid
for ax in axes:
    ax.grid(False)

plt.show()

In [ ]:
def get_stats(colname):
    return pd.concat(
        [
            df.loc[df["churn"] == "Sim", [colname]]
            .select_dtypes(include="number")
            .describe(
                percentiles=[
                    0.25,
                    0.5,
                    0.75,
                    0.9,
                    0.95,
                    0.99,
                ]
            )
            .T.assign(
                skewness=df.loc[df["churn"] == "Sim", [colname]].skew().round(2),
                kurtosis=df.loc[df["churn"] == "Sim", [colname]].kurt().round(2),
                desc="Não renovou",
            ),
            df.loc[df["churn"] == "Não", [colname]]
            .select_dtypes(include="number")
            .describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])
            .T.assign(
                skewness=df.loc[df["churn"] == "Não", [colname]].skew().round(2),
                kurtosis=df.loc[df["churn"] == "Não", [colname]].kurt().round(2),
                desc="Renovou",
            ),
            df.loc[:, [colname]]
            .select_dtypes(include="number")
            .describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])
            .T.assign(
                skewness=df.loc[:, [colname]].skew().round(2),
                kurtosis=df.loc[:, [colname]].kurt().round(2),
                desc="Todos",
            ),
        ],
        axis=0,
    )


display(
    HTML(
        "<p style='font-weight:bold'>Churn por receita mensal</p>"
        + get_stats("receita_mensal").to_html()
    )
)

display(
    HTML(
        "<p style='font-weight:bold'>Churn por receita total</p>"
        + get_stats("receita_total").to_html()
    )
)

display(
    HTML(
        "<p style='font-weight:bold'>Churn por meses de permanência</p>"
        + get_stats("meses_de_permanencia").to_html()
    )
)

Vemos nas tabelas de estatística descritiva que há clientes com "meses de permanência" igual a 0, e como mencionamos na seção 1, há clientes sem receita total. Pode ser possível que esses dados se refiram a clientes que acabaram de adquirir o serviço, ou seja, não tiveram tempo de permanência suficiente para ter receita total.

In [ ]:
df.loc[
    (df["receita_total"].isna()) | (df["meses_de_permanencia"] == 0),
    [
        "tipo_de_empresa",
        "fundacao_da_empresa",
        "possui_mais_de_um_socio",
        "funcionarios",
        "receita_total",
        "meses_de_permanencia",
        "churn",
    ],
]

In [ ]:
df.loc[df["meses_de_permanencia"] == 1].assign(
    receita_total_corrigida=df["receita_mensal"] * df["meses_de_permanencia"]
)[
    [
        "receita_mensal",
        "meses_de_permanencia",
        "receita_total_corrigida",
        "receita_total",
    ]
].describe()

Em seguida, verificamos a correlação entre as variáveis com a variável dependente `Churn`. As variáveis `meses de permanência` e `contrato` são as que possuem maior correlação com a variável `Churn`. Foi algo que notamos ao observar o histograma nas análises anteriores, onde clientes com menos tempo de permanência e contrato mês-a-mês foram os que mais deram churn.

In [ ]:
corr = (
    df.assign(
        contrato=df["contrato"].cat.codes,
        emite_boletos=df["emite_boletos"].cat.codes,
        forma_de_pagamento=df["forma_de_pagamento"].cat.codes,
        faz_conciliacao_bancaria=df["faz_conciliacao_bancaria"].cat.codes,
        funcionarios=df["funcionarios"].cat.codes,
        possui_mais_de_um_socio=df["possui_mais_de_um_socio"].cat.codes,
        frequencia_de_pagamento=df["frequencia_de_pagamento"].cat.codes,
        churn=df["churn"].cat.codes,
    )
    .select_dtypes(include="number")
    .corr()
)


# Gera uma máscara para o triângulo superior
mask = np.triu(np.ones_like(corr, dtype=bool))

# Define uma figura do matplotlib
ax = plt.figure(figsize=(10, 10))

# Gera um colormap divergente
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Desenha um heatmap com a máscara e a correta proporção
sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=1,
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
    annot=True,
)

plt.show()

## 3. Salvando o conjunto de dados

Por fim, salvamos o conjunto de dados pré-processado para ser utilizado na modelagem.

In [ ]:
df[
    [
        "emite_boletos",
        "possui_mais_de_um_socio",
        "utiliza_servicos_financeiros",
        "churn",
    ]
] = (
    df[
        [
            "emite_boletos",
            "possui_mais_de_um_socio",
            "utiliza_servicos_financeiros",
            "churn",
        ]
    ]
    .astype("str")
    .replace({"Sim": "1", "Não": "0"})
    .astype("int")
    .astype("bool")
)

Armazenamos o conjunto de dados no *BigQuery*, um serviço de armazenamento de dados na nuvem do Google.

In [ ]:
pandas_gbq.to_gbq(
    df,
    destination_table="customer_churn.customer_churn_data",
    project_id="ca-churn-project",
    if_exists="replace",
)